In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# ==========================================
# 1. PHYSICS SETUP (Kidney Context)
# ==========================================
ALPHA_CORTEX = 0.01
ALPHA_MEDULLA = 0.02
H_COEFF = 5.0
T_INF = -1.0
T_INITIAL = 1.0

In [ ]:
# ==========================================
# 2. ADVANCED ARCHITECTURE: ResNet PINN
# ==========================================
class ResNetBlock(nn.Module):
    """A Residual Block that allows deep networks to learn physics."""
    def __init__(self, neurons):
        super(ResNetBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(neurons, neurons),
            nn.SiLU(), # SiLU (Swish) is better for PINNs than Tanh
            nn.Linear(neurons, neurons),
            nn.SiLU()
        )
    
    def forward(self, x):
        return x + self.layer(x) # The "Skip Connection"

class AdvancedPINN(nn.Module):
    def __init__(self, layers=4, neurons=50):
        super(AdvancedPINN, self).__init__()
        
        # Input Layer (x, y, t)
        self.input_layer = nn.Sequential(
            nn.Linear(3, neurons),
            nn.SiLU()
        )
        
        # Hidden ResNet Layers
        self.res_blocks = nn.ModuleList(
            [ResNetBlock(neurons) for _ in range(layers)]
        )
        
        # Output Layer
        self.output_layer = nn.Linear(neurons, 1)

    def forward(self, x, y, t):
        # Concatenate inputs
        inputs = torch.cat([x, y, t], dim=1)
        out = self.input_layer(inputs)
        
        # Pass through residual blocks
        for block in self.res_blocks:
            out = block(out)
            
        return self.output_layer(out)

In [ ]:
# ==========================================
# 3. GEOMETRY & LOSS (Kidney Logic)
# ==========================================
def get_kidney_points(n_points):
    """Generates random points inside the bean shape."""
    points_x, points_y = [], []
    count = 0
    while count < n_points:
        xt = torch.rand(n_points, 1) * 2 - 1
        yt = torch.rand(n_points, 1) * 2 - 1
        # Kidney Shape Logic
        y_bent = yt - 0.2 * torch.sin(3 * xt)
        mask = ((xt / 0.8)**2 + (y_bent / 0.5)**2) <= 1.0
        
        valid_x = xt[mask].unsqueeze(1)
        valid_y = yt[mask].unsqueeze(1)
        points_x.append(valid_x)
        points_y.append(valid_y)
        count += len(valid_x)
    return torch.cat(points_x)[:n_points], torch.cat(points_y)[:n_points]

def physics_loss(model, x, y, t, bc_weight=1.0):
    """Calculates PDE Residual with adjustable BC weight."""
    x.requires_grad = True; y.requires_grad = True; t.requires_grad = True
    u = model(x, y, t)
    
    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, torch.ones_like(u_y), create_graph=True)[0]
    
    # Heterogeneous Alpha (Cortex vs Medulla)
    r = torch.sqrt(x**2 + y**2)
    alpha = ALPHA_CORTEX + (ALPHA_MEDULLA - ALPHA_CORTEX) * torch.sigmoid(10*(0.4-r))
    
    # PDE Loss
    loss_pde = torch.mean((u_t - alpha * (u_xx + u_yy)) ** 2)
    
    # Boundary Loss (Simplified Cooling)
    # Sampling boundary points implicitly via domain edge approximation
    theta = torch.rand(len(x), 1) * 2 * np.pi
    xb = 0.8 * torch.cos(theta)
    yb = 0.5 * torch.sin(theta) + 0.2 * torch.sin(3 * 0.8 * torch.cos(theta))
    tb = torch.rand(len(x), 1)
    u_bound = model(xb, yb, tb)
    target = T_INF + (T_INITIAL - T_INF) * torch.exp(-H_COEFF * tb)
    loss_bc = torch.mean((u_bound - target) ** 2)
    
    return loss_pde + (bc_weight * loss_bc)

In [ ]:
# ==========================================
# 4. HYPERPARAMETER TUNING ENGINE
# ==========================================
def run_experiment(config):
    """Trains a model for a short duration with specific config."""
    print(f"Testing Config: {config}")
    model = AdvancedPINN(layers=config['layers'], neurons=config['neurons'])
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    
    # Short training run for validation
    x_p, y_p = get_kidney_points(1000)
    t_p = torch.rand(1000, 1)
    
    final_loss = 0
    for epoch in range(1000): # Short run
        optimizer.zero_grad()
        loss = physics_loss(model, x_p, y_p, t_p, bc_weight=config['bc_weight'])
        loss.backward()
        optimizer.step()
        final_loss = loss.item()
        
    return final_loss, model

In [ ]:
# HYPERPARAMETER GRID
param_grid = {
    'lr': [0.01, 0.001, 0.0005],
    'layers': [2, 4, 6],
    'neurons': [32, 64, 128],
    'bc_weight': [1.0, 5.0, 10.0] # How important is the boundary?
}

print("🔬 STARTING HYPERPARAMETER TUNING (Random Search)...")
best_loss = float('inf')
best_config = {}
best_model = None

# Run 5 Random Experiments
for i in range(5):
    config = {k: random.choice(v) for k, v in param_grid.items()}
    loss, model = run_experiment(config)
    print(f"  > Experiment {i+1} Loss: {loss:.6f}")
    
    if loss < best_loss:
        best_loss = loss
        best_config = config
        best_model = model # Save the winner

print(f"\n🏆 WINNER: {best_config} with Loss: {best_loss:.6f}")
print("🚀 Training Winner to Full Convergence (4000 more epochs)...")

In [ ]:
# ==========================================
# 5. FINAL TRAINING (The Winner)
# ==========================================
optimizer = torch.optim.Adam(best_model.parameters(), lr=best_config['lr'])
x_phys, y_phys = get_kidney_points(3000)
t_phys = torch.rand(3000, 1)

loss_history = []
for epoch in range(4001):
    optimizer.zero_grad()
    loss = physics_loss(best_model, x_phys, y_phys, t_phys, bc_weight=best_config['bc_weight'])
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    
    if epoch % 1000 == 0:
        print(f"Final Training Epoch {epoch}: Loss = {loss.item():.6f}")

In [ ]:
# ==========================================
# 6. VISUALIZATION (Tuning Results)
# ==========================================
plt.figure(figsize=(12, 5))

# Plot 1: The Kidney Map
plt.subplot(1, 2, 1)
x_grid = np.linspace(-1, 1, 100)
y_grid = np.linspace(-1, 1, 100)
X, Y = np.meshgrid(x_grid, y_grid)
X_flat = torch.tensor(X.flatten()[:, None], dtype=torch.float32)
Y_flat = torch.tensor(Y.flatten()[:, None], dtype=torch.float32)
T_flat = torch.ones_like(X_flat) * 0.5 # Snapshot at t=0.5

best_model.eval()
with torch.no_grad():
    U_pred = best_model(X_flat, Y_flat, T_flat).numpy().reshape(X.shape)

Y_bent = Y - 0.2 * np.sin(3 * X)
mask = ((X / 0.8)**2 + (Y_bent / 0.5)**2) <= 1.0
U_masked = np.where(mask, U_pred, np.nan)

plt.imshow(U_masked, interpolation='bilinear', cmap='RdBu_r', origin='lower', extent=[-1, 1, -1, 1])
plt.colorbar(label='Temp')
plt.title(f'Best Model: {best_config["layers"]} Layers, {best_config["neurons"]} Neurons')

In [ ]:
# Plot 2: Convergence History
plt.subplot(1, 2, 2)
plt.plot(loss_history, 'g-', label='Winner Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Physics Loss (Log Scale)')
plt.title('Optimization Convergence')
plt.legend()

plt.tight_layout()
plt.savefig('advanced_tuning_results.png', dpi=300)
plt.show()